# Spooky Books Author Prediction - v9
Simplified Stratified K-Fold with tuned Logistic Regression and Naive Bayes, combined via weighted soft voting.

## 1. Import Libraries

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import loguniform
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import log_loss
from scipy.sparse import hstack

## 2. Load & Encode Data

In [2]:
train = pd.read_csv('./train/train.csv')
test  = pd.read_csv('./test/test.csv')
X_text = train['text']
y = train['author']
X_test_text = test['text']

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_enc = le.fit_transform(y)
print('Classes:', le.classes_)
print('Train shape:', train.shape, 'Test shape:', test.shape)

Classes: ['EAP' 'HPL' 'MWS']
Train shape: (19579, 3) Test shape: (8392, 2)


## 3. Combined TF-IDF Features

In [3]:
word_tf = TfidfVectorizer(ngram_range=(1,2), max_features=15000, stop_words='english')
char_tf = TfidfVectorizer(analyzer='char_wb', ngram_range=(3,5), max_features=5000)

X_word = word_tf.fit_transform(X_text)
X_char = char_tf.fit_transform(X_text)
X = hstack([X_word, X_char])

X_test = hstack([word_tf.transform(X_test_text), char_tf.transform(X_test_text)])

## 4. Hyperparameter Tuning

In [4]:
# Tune Logistic Regression
lr = LogisticRegression(solver='saga', multi_class='multinomial', max_iter=2000, random_state=42, n_jobs=-1)
lr_search = RandomizedSearchCV(
    lr, {'C': loguniform(1e-2, 1e2)}, n_iter=10,
    scoring='neg_log_loss', cv=5, n_jobs=-1, random_state=42
)
lr_search.fit(X, y_enc)
print('LR best C:', lr_search.best_params_)

# Tune MultinomialNB
nb = MultinomialNB()
nb_search = RandomizedSearchCV(
    nb, {'alpha': np.linspace(0.1, 1.0, 10)}, n_iter=10,
    scoring='neg_log_loss', cv=5, n_jobs=-1, random_state=42
)
nb_search.fit(X, y_enc)
print('NB best alpha:', nb_search.best_params_)

C:\Users\LENOVO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


LR best C: {'C': np.float64(6.79657809075816)}
NB best alpha: {'alpha': np.float64(0.1)}


## 5. Stratified K-Fold Ensemble

In [5]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
test_preds = np.zeros((X_test.shape[0], len(le.classes_)))
val_losses = []

# Calculate weights inversely proportional to CV log-loss
lr_loss = -lr_search.best_score_
nb_loss = -nb_search.best_score_
w_lr = nb_loss / (lr_loss + nb_loss)
w_nb = lr_loss / (lr_loss + nb_loss)
print(f'Weights -> LR: {w_lr:.2f}, NB: {w_nb:.2f}')

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y_enc)):
    X_tr, X_val = X[train_idx], X[val_idx]
    y_tr, y_val = y_enc[train_idx], y_enc[val_idx]

    lr = LogisticRegression(C=lr_search.best_params_['C'], solver='saga', \
                           multi_class='multinomial', max_iter=2000, random_state=42, n_jobs=-1)
    nb = MultinomialNB(alpha=nb_search.best_params_['alpha'])
    
    lr.fit(X_tr, y_tr)
    nb.fit(X_tr, y_tr)

    p_lr = lr.predict_proba(X_val)
    p_nb = nb.predict_proba(X_val)
    p_val = w_lr * p_lr + w_nb * p_nb
    val_loss = log_loss(y_val, p_val)
    val_losses.append(val_loss)
    print(f'Fold {fold+1} Log Loss: {val_loss:.4f}')

    p_test_lr = lr.predict_proba(X_test)
    p_test_nb = nb.predict_proba(X_test)
    test_preds += (w_lr * p_test_lr + w_nb * p_test_nb) / skf.n_splits

print(f'Average CV Log Loss: {np.mean(val_losses):.4f}')

Weights -> LR: 0.50, NB: 0.50


C:\Users\LENOVO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Fold 1 Log Loss: 0.3903


C:\Users\LENOVO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Fold 2 Log Loss: 0.3678


...

Fold 3 Log Loss: 0.3898


C:\Users\LENOVO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Fold 4 Log Loss: 0.3845


C:\Users\LENOVO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Fold 5 Log Loss: 0.3813
Average CV Log Loss: 0.3827


## 6. Submission

In [6]:
submission = pd.DataFrame(test_preds, columns=le.classes_)
submission.insert(0, 'id', test['id'])
submission.to_csv('./sample_submission/submission_v9.csv', index=False)
print('Saved: submission_v9.csv')

Saved: submission_v9.csv
